In [311]:
# Importing the necessary libraries
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)

In [312]:
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.feature_selection import RFE

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

## **Initial Exploration of the data**

In [313]:
#Importing the dataset 
pd.set_option("display.max_columns",None)
df = pd.read_csv("../input/houing/train.csv")

In [314]:
#Looking first few rows of dataset
df.head()

In [315]:
#Checking the dimensions of the dataset
df.shape

#### Dealing with Null Values

In [316]:
#Looking the number of nulls in the dataframe
(round((df.isnull().sum())/(len(df.index)) * 100,2)).sort_values(ascending= False)

Dropping columns which have more 80% null values. It is seen from the above output that Alley,PoolQC,Fence,MiscFeature are having 80% null values and there willeliminated from the dataset

In [317]:
#The columns with highest null value are dropped
df.columns

In [318]:
#To handle the missing values of other columns, let us look at the type of numbers they hold
df.info()

Replacing object type variables with mode and float or int type column with median

In [319]:
round((df.isnull().sum())/(len(df.index)) * 100,2).sort_values(ascending= False)

In [320]:
#addressing NaN values based on data dictionary

# In column 'PoolQC' (Pool quality), NaN stands for No Pool
df['PoolQC'] = df['PoolQC'].fillna('No_Pool')

# In column 'MiscFeature' (Miscellaneous Features), NaN stands for None, meaning the house has no miscellaneous features.
df['MiscFeature'] = df['MiscFeature'].fillna('None')

# In column 'Alley', NaN stands for No Alley Access as per the data dictionary
df['Alley'] = df['Alley'].fillna('No_Alley_Access')

# In column 'Fence' (Fence Quality), NaN stands for No Fence as per the data dictionary
df['Fence'] = df['Fence'].fillna('No_Fence')

# In column 'FireplaceQu' (FireplaceQu Quality), NaN stands for No Fireplace as per the data dictionary
df['FireplaceQu'] = df['FireplaceQu'].fillna('No_Fireplace')

# LotFrontage stands for Linear feet of street connected to property, there is no explanation to impute this in data dictionary
# Let's consider imputing it with median of the lotFrontage of houses in the same neighbourhood

# Group data by neighborhood and impute missing value with median LotFrontage of all the neighborhood
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

# In column 'GarageYrBlt' (Gargae Year Built), NaN stands for houses with no garage, let's impute with 0
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)

# 'GarageType', 'GarageFinish', 'GarageQual' (Garage Quality) and 'GarageCond'(Garage Condition)
# Missing values signify no garage as per data dictionary.let's impute NaN values here with No Garage
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    df[col] = df[col].fillna('No_Garage')

#BsmtFinType1, BsmtFinType2 (Rating of basement finished area),
#BsmtExposure (Basement Exposure), BsmtQual(Basement Quality), BsmtCond (Basement Conidtion)
#These are all parameter related to basement. A NaN value probably signifies that the house does not have a basement.
for col in ('BsmtFinType1', 'BsmtFinType2', 'BsmtExposure', 'BsmtQual','BsmtCond'):
    df[col] = df[col].fillna('No_Basement')
    
# In column 'MasVnrType' (Masonry veneer type), let's impute it with mode "None"
df['MasVnrType'] = df['MasVnrType'].fillna('None')

# In column 'MasVnrType' (Masonry veneer type), let's impute it with mode 0 corresponding to None
df['MasVnrArea'] = df['MasVnrArea'].fillna(0)

# In column 'Electrical' (Electrical system), let's impute NaN with "Other"
df['Electrical'] = df['Electrical'].fillna("Other")

In [321]:
#checking percentage of null values in each column

df_missing=pd.DataFrame((round(100*(df.isnull().sum()/len(df.index)), 2)), columns=['missing'])
df_missing.sort_values(by=['missing'], ascending=False).head(20)

In [322]:
#Converting the values of MSSubClass to the corresponding numeric values as mentioned in the data dictionary
df=df.replace({'MSSubClass' : { 20 : '1-STORY 1946 & NEWER ALL STYLES', 
                                          30:'1-STORY 1945 & OLDER',
                                          40:'1-STORY W/FINISHED ATTIC ALL AGES',
                                          45:'1-1/2 STORY - UNFINISHED ALL AGES',
                                          50:'1-1/2 STORY FINISHED ALL AGES',
                                          60:'2-STORY 1946 & NEWER',
                                          70:'2-STORY 1945 & OLDER',
                                          75:'2-1/2 STORY ALL AGES',
                                          80:'SPLIT OR MULTI-LEVEL',
                                          85:'SPLIT FOYER',
                                          90:'DUPLEX - ALL STYLES AND AGES',
                                         120:'1-STORY PUD (Planned Unit Development) - 1946 & NEWER',
                                         150:'1-1/2 STORY PUD - ALL AGES',
                                         160:'2-STORY PUD - 1946 & NEWER',
                                         180:'PUD - MULTILEVEL - INCL SPLIT LEV/FOYER',
                                         190:'2 FAMILY CONVERSION - ALL STYLES AND AGES'}})

In [323]:
df.select_dtypes('object').nunique()

In [324]:
#Using label encoding to ctegorize the values in columns that has heirarchy in its vales
# label encode ordinal features where there is order in categories

df = df.replace({  "Alley":        {"No_Alley_Access" : 0, "Grvl" : 1, "Pave" : 2},
                       "BsmtCond":     {"No_Basement" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "BsmtExposure": {"No_Basement" : 0, "No" : 2, "Mn" : 2, "Av": 3, 
                                        "Gd" : 4},
                       "BsmtFinType1": {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, 
                                        "BLQ" : 4, "ALQ" : 5, "GLQ" : 6},
                       "BsmtFinType2": {"No_Basement" : 0, "Unf" : 1, "LwQ": 2, "Rec" : 3, 
                                        "BLQ" : 4, 
                                         "ALQ" : 5, "GLQ" : 6},
                       "BsmtQual":     {"No_Basement" : 0, "Po" : 1, "Fa" : 2, "TA": 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "CentralAir":   {"None" : 0, "N" : 1, "Y" : 2},
                       "ExterCond":    {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, 
                                        "Gd": 4, "Ex" : 5},
                       "ExterQual":    {"None" : 0, "Po" : 1, "Fa" : 2, "TA": 3, 
                                        "Gd": 4, "Ex" : 5},
                       "Fence":        {"No_Fence" : 0, "MnWw" : 1, "GdWo" : 2, "MnPrv": 3, 
                                        "GdPrv" : 4},
                       "FireplaceQu":  {"No_Fireplace" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "Functional":   {"None" : 0, "Sal" : 1, "Sev" : 2, "Maj2" : 3, 
                                        "Maj1" : 4, "Mod": 5, "Min2" : 6, "Min1" : 7, 
                                        "Typ" : 8},
                       "GarageCond":   {"No_Garage" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "GarageQual":   {"No_Garage" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "GarageFinish": {"No_Garage" : 0, "Unf" : 1, "RFn" : 2, "Fin" : 3},
                       "HeatingQC":    {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "KitchenQual":  {"None" : 0, "Po" : 1, "Fa" : 2, "TA" : 3, 
                                        "Gd" : 4, "Ex" : 5},
                       "LandContour":  {"None" : 0, "Low" : 1, "HLS" : 2, "Bnk" : 3, 
                                        "Lvl" : 4},
                       "LandSlope":    {"None" : 0, "Sev" : 1, "Mod" : 2, "Gtl" : 3},
                       "LotShape":     {"None" : 0, "IR3" : 1, "IR2" : 2, "IR1" : 3, 
                                        "Reg" : 4},
                       "PavedDrive":   {"None" : 0, "N" : 0, "P" : 1, "Y" : 2},
                       "PoolQC":       {"No_Pool" : 0, "Fa" : 1, "TA" : 2, "Gd" : 3, 
                                        "Ex" : 4},
                       "Street":       {"None" : 0, "Grvl" : 1, "Pave" : 2},
                       "Utilities":    {"None" : 0, "ELO" : 1, "NoSeWa" : 2, "NoSewr" : 3, 
                                        "AllPub" : 4}}
                     )

df.BsmtCond = df.BsmtCond.astype(int)

In [325]:
#changing months to categorical
import calendar

df['MonthSold'] = df['MoSold'].apply(lambda x: calendar.month_name[x])
df=df.drop(['MoSold'], axis=1)

In [326]:
#changing data type of Gararge yr built to int from float
df['GarageYrBlt'] = df['GarageYrBlt'].astype(int)

In [327]:
#DERIVED VARIABLES which might make more sense than year

df['Age'] = df['YrSold'] - df['YearBuilt']
df['Remod_Age'] = df['YrSold'] - df['YearRemodAdd']
df['Garage_Age'] = df['YrSold'] - df['GarageYrBlt']
df.drop(['YearBuilt','YearRemodAdd','GarageYrBlt','YrSold'],1, inplace = True)

#### Outlier Treatment

In [328]:
#creating list of categorical and numeric columns
object_columns = df.select_dtypes('object').columns
numeric_columns = df.select_dtypes(['float64','int64']).columns

In [329]:
#The Highest number of outliers is in BsmtFinSF2 which corresponds to 3.42%. If the outlier count count is less than 1% of the dataset it will be dropped
for cols in df[numeric_columns]:
    upper_limit = (df[cols].mean() + 3*df[cols].std())
    lower_limit = (df[cols].mean() - 3*df[cols].std())
    outlier_count = round(df[cols].loc[(df[cols] >= upper_limit) | (df[cols] <= lower_limit)].count()/(len(df.index))*100,2)
    if outlier_count < 1:
        df = df[(df[cols] <= upper_limit) & (df[cols] >= lower_limit)]

In [330]:
df.shape

Visualizing the data

In [331]:
df_num = df[numeric_columns]
for z in numeric_columns[:-1]:
    sns.pairplot(data = df_num,x_vars = z,y_vars = 'SalePrice')
    plt.show()

In [332]:
#Dropping the ID column since it does not have significance to the model
df = df.drop(['Id'],axis=1)
df_catcols = df.select_dtypes(include=['object'])
df_catcols.head()

### Handling the categorical variables

In [333]:
#Creating dummy variables for the categoricle column and dropping the original variables
cat_cols = [cols for cols in df.select_dtypes('object').columns]
df_dummies = pd.get_dummies(df[cat_cols],prefix = cat_cols,drop_first = True)
df = pd.concat([df,df_dummies],axis = 1)
df = df.drop(list(df[cat_cols].columns), axis=1)
df.shape

In [334]:
df.head(5)

### Creating Test and Train data split

In [335]:
#train_test_split
df_train,df_test=train_test_split(df,train_size=0.70, random_state=100)
df_train.shape

In [336]:
y_train = np.log(df_train.SalePrice)
X_train = df_train.drop("SalePrice",1)

y_test= np.log(df_test.SalePrice)
X_test = df_test.drop("SalePrice",1)

In [337]:
num_vars=X_train.select_dtypes(include=['int64','float64']).columns

In [338]:
num_vars

In [339]:
# Standardising the dataset
scaler = StandardScaler()
X_train[num_vars] = scaler.fit_transform(X_train[num_vars])
X_test[num_vars] = scaler.transform(X_test[num_vars])

In [340]:
# Building the linear regression model
lm=LinearRegression()
lm.fit(X_train,y_train)

rfe = RFE(lm,20)
rfe=rfe.fit(X_train,y_train)

col=X_train.columns[rfe.support_]
col

In [341]:
X_train_new=X_train[col]

X_train_new = sm.add_constant(X_train_new)

#create first model
lr=sm.OLS(y_train,X_train_new)

#fit the model
lr_model=lr.fit()

lr_model.summary()

In [342]:
#RIDGE REGULARIZATION

# Using grid search to find the best value of alpha for the model


params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100]}


ridge = Ridge()

# cross validation
folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            
model_cv.fit(X_train, y_train)

In [343]:
print(model_cv.best_params_)
print(model_cv.best_score_)

In [344]:
cv_results = pd.DataFrame(model_cv.cv_results_)
cv_results = cv_results[cv_results['param_alpha']<=100]
cv_results.head(5)

In [345]:
# plotting mean test and train scoes with alpha 
cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')
plt.figure(figsize=(16,5))

# plotting
plt.plot(cv_results['param_alpha'], cv_results['mean_train_score'])
plt.plot(cv_results['param_alpha'], cv_results['mean_test_score'])
plt.xlabel('alpha')
plt.ylabel('Negative Mean Absolute Error')
plt.title("Negative Mean Absolute Error and alpha")
plt.legend(['train score', 'test score'], loc='upper right')
plt.show()

In [346]:
#final ridge model
alpha = 10
ridge = Ridge(alpha=alpha)

ridge.fit(X_train, y_train)
ridge.coef_

In [347]:
#lets predict the R-squared value of test and train data
y_train_pred = ridge.predict(X_train)
print(metrics.r2_score(y_true=y_train, y_pred=y_train_pred))

In [348]:
y_test_pred = ridge.predict(X_test)
print(metrics.r2_score(y_true=y_test, y_pred=y_test_pred))

In [349]:
from sklearn.metrics import mean_squared_error
print ('RMSE is: \n', mean_squared_error(y_test, y_test_pred))

In [350]:
# Ridge model parameters
model_parameters_1 = list(ridge.coef_)
model_parameters_1.insert(0, ridge.intercept_)
model_parameters_1 = [round(x, 3) for x in model_parameters_1]
cols = X_train.columns
cols = cols.insert(0, "constant")
Vars_coeff = list(zip(cols, model_parameters_1))
sorted(Vars_coeff, key = lambda x: x[1])

In [351]:
#lasso
params = {'alpha': [0.00005, 0.0001, 0.001, 0.008, 0.01]}
lasso = Lasso()

# cross validation
model_cv_l = GridSearchCV(estimator = lasso, 
                        param_grid = params, 
                        scoring= 'neg_mean_absolute_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)            

model_cv_l.fit(X_train, y_train)

In [352]:
# cv results
cv_results_l = pd.DataFrame(model_cv_l.cv_results_)

In [353]:
#checking the value of optimum number of parameters
print(model_cv_l.best_params_)
print(model_cv_l.best_score_)

In [354]:
#final lasso model
alpha = 0.001

lasso = Lasso(alpha=alpha)
        
lasso.fit(X_train, y_train) 

In [355]:
#lets predict the R-squared value of test and train data
y_train_pred = lasso.predict(X_train)
print(metrics.r2_score(y_true=y_train, y_pred=y_train_pred))

In [356]:
#lets predict the R-squared value of test and train data
y_test_pred = lasso.predict(X_test)
print(metrics.r2_score(y_true=y_test, y_pred=y_test_pred))

In [357]:
from sklearn.metrics import mean_squared_error
print ('RMSE is: \n', mean_squared_error(y_test, y_test_pred))


In [358]:
# Lasso model parameters
model_parameters_1 = list(lasso.coef_)
model_parameters_1.insert(0, lasso.intercept_)
model_parameters_1 = [round(x, 3) for x in model_parameters_1]
cols = X_train.columns
cols = cols.insert(0, "constant")
list(zip(cols, model_parameters_1))
vars_coeff = list(zip(cols, model_parameters_1))
sorted(vars_coeff, key = lambda x: x[1])

## Observations


### Ridge Regression 
Alpha = 10    
R-Squared    
    Train - 0.94    
    Test - 0.88    
RMSE - 0.015579464291382315    

#### Top Variables in Lasso Regression Model  and their corresponding weights are as follows  
 
1.Age, -0.059  
2.MSSubClass_2-STORY PUD - 1946 & NEWER, -0.051  
3.Neighborhood_MeadowV, -0.042  
4.MSSubClass_1-STORY 1945 & OLDER, -0.037  
5.Heating_Grav, -0.035  
6.Neighborhood_OldTown, -0.032  
7.BldgType_Twnhs, -0.032  
8.Neighborhood_IDOTRR, -0.027  
9.Neighborhood_Mitchel, -0.027  
10.Neighborhood_Gilbert, -0.026  

 

### Lasso Regression
Alpha = 0.001  
R-Squared  
Train - 0.93     
Test - 0.88    
RMSE - 0.015942169442860383  

#### Top Variables in Lasso Regression Model  and their corresponding weights are as follows  
1.Age -0.062  
2.MSSubClass_2-STORY PUD - 1946 & NEWER -0.044  
3.MSSubClass_1-STORY 1945 & OLDER -0.032  
4.SaleType_WD -0.016  
5.BsmtCond -0.013  
6.KitchenAbvGr -0.012  
7.Remod_Age -0.012  
8.MonthSold_November -0.012  
9.BldgType_Twnhs -0.008  
10.Neighborhood_OldTown -0.006  

  
### Comparing the evaluation metrics for two models it is safe to infer that the Ridge model is performing better than the Lasso model  



